# Code Execution Scorer

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JudgmentLabs/judgment-cookbook/blob/refactor/HumanEval_Code_Scorer.ipynb
)

In this notebook, you will learn how to create **custom code-based scorers** using the **judgeval** library. This implementation demonstrates how to evaluate code generation using OpenAI's HumanEval benchmark and their sandboxed execution environment.

You will generate code using LLMs, create a custom scorer that leverages OpenAI's sandboxed environment, and evaluate it against the HumanEval benchmark dataset using functional correctness testing.

In [ ]:
# Installations
!pip install human-eval datasets openai judgeval


To run this notebook and train a WikiRacer Agent, select **Runtime* -> Run All*

In [ ]:
# set api keys
import os
from dotenv import load_dotenv

os.environ['OPENAI_API_KEY'] = ...
os.environ["JUDGMENT_API_KEY"] = ...
os.environ["JUDGMENT_ORG_ID"] = ...

load_dotenv()

In [ ]:
from judgeval import JudgmentClient
from judgeval.dataset import Dataset
from judgeval.scorers.example_scorer import ExampleScorer
from judgeval.data import Example
from datasets import load_dataset
from human_eval.execution import check_correctness
from openai import AsyncOpenAI
from typing import Dict, Any
import asyncio

In [2]:
# Initialize clients
judgment = JudgmentClient()
client = AsyncOpenAI()

## 1. Understanding HumanEval

HumanEval is a benchmark dataset created by OpenAI for evaluating code generation models. Introduced in the paper ["Evaluating Large Language Models Trained on Code"](https://arxiv.org/abs/2107.03374), it contains 164 Python programming problems designed to test functional correctness.

### What HumanEval Contains
Each problem includes:
- **Function signature and docstring**: The problem description
- **Test cases**: Automated tests to verify correctness  
- **Canonical solution**: Reference implementation
- **Entry point**: Function name to test

### How HumanEval Evaluates Code
HumanEval evaluates model outputs by dynamically building a Python program that stitches together the **Function signature and docstring**, the **model’s generated solution**, and the **test cases**. This combined program is then executed in a sandbox to verify whether the generated code passes all test cases.

The ```check_correctness``` function orchestrates this process: assembles the prompt, generated solution, and tests into a single program, and executes the script in a sandboxed environment.

```python
# Construct the check program and run it.
check_program = (
    problem["prompt"] +      # Function signature + docstring
    completion +            # Generated code
    "\n" +
    problem["test"] +       # Test cases
    "\n" +
    f"check({problem['entry_point']})"  # Call the test function
)

...

# WARNING: This executes untrusted model-generated code
exec(check_program, exec_globals)
```

The evaluation is **pass/fail**: if all test cases pass without exceptions, the code is correct. If any test fails or the code crashes, it's incorrect.

## Code Generation Function

Next, we’ll implement a function that, given a HumanEval problem, queries an LLM to produce a candidate implementation. The function is written with `async/await` so multiple problems can be evaluated in parallel, significantly reducing total runtime.

In [3]:
async def generate_code(problem: Dict[str, Any]) -> str:
    """Generate code using LLM for a given HumanEval problem."""
    prompt = problem["prompt"]
    
    response = await client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are an expert Python programmer. Write ONLY the Python function code that solves the given problem. Do not include any markdown formatting, explanations, or code blocks. Return only the raw Python code."},
            {"role": "user", "content": prompt}
        ],
    )
    
    generated_code = response.choices[0].message.content
    
    return generated_code

## Custom Code Execution Scorer

We'll create a custom scorer using judgeval that integrates HumanEval's sandboxed code execution. The scorer uses the `check_correctness` function to evaluate whether generated code passes the test cases.

The scorer runs the generated code in a sandboxed environment and checks if all tests pass. If they do, it assigns a score of 1.0. If any test fails or the code crashes, it assigns a score of 0.0.

In [4]:
class HumanEvalCodeExecutionScorer(ExampleScorer):
    """
    A scorer for evaluating code generation using functional correctness.
    
    This scorer uses the human_eval.execution.check_correctness function
    to run generated code against test cases and determine if it passes.
    
    Attributes:
        name (str): The name of the scorer
    """
    name: str = "HumanEval Code Execution Scorer"

    async def a_score_example(self, example: Example) -> None:
        """
        Score an example by running the generated code against test cases.
        
        This method uses check_correctness to execute the generated code
        in a sandboxed environment and check if it passes all test cases.
        
        Args:
            example (HumanEvalExample): The example containing the problem and generated code
            
        Returns:
            float: The score (1.0 if all tests pass, 0.0 otherwise)
        """
        # Create problem dict in the format expected by check_correctness
        problem = {
            "task_id": example.task_id,
            "prompt": example.prompt,
            "test": example.test,
            "entry_point": example.entry_point
        }
        
        # Use check_correctness to evaluate the generated code
        result = check_correctness(
            problem=problem,
            completion=example.generated_code,
            timeout=3.0
        )
        
        # Set score based on whether tests passed
        if result["passed"]:
            self.score = 1.0
            self.reason = "All test cases passed"
        else:
            self.score = 0.0
            self.reason = f"Test failed: {result['result']}"
        
        return self.score

## Load HumanEval Dataset

Now let's load the HumanEval dataset from Hugging Face and examine its structure.


In [5]:
# Load the HumanEval dataset
print("📊 Loading HumanEval dataset...")
dataset = load_dataset("openai/openai_humaneval")
print(f"   Found {len(dataset['test'])} problems")

# Examine the structure of a single problem
example_problem = dataset["test"][0]
print("\n📋 Example problem structure:")
print(f"   Task ID: {example_problem['task_id']}")
print(f"   Entry Point: {example_problem['entry_point']}")
print(f"   Prompt length: {len(example_problem['prompt'])} characters")
print(f"   Test length: {len(example_problem['test'])} characters")

print("\n📝 Sample prompt:")
print(example_problem['prompt'][:200] + "...")

📊 Loading HumanEval dataset...
   Found 164 problems

📋 Example problem structure:
   Task ID: HumanEval/0
   Entry Point: has_close_elements
   Prompt length: 348 characters
   Test length: 531 characters

📝 Sample prompt:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given thr...


Generate code responses for each problem in the HumanEval benchmark and create example objects to upload into judgeval.

In [6]:
print("\n🤖 Generating code...")
problems = list(dataset["test"].select(range(164)))

# Generate all code in parallel
generated_codes = await asyncio.gather(*[
    generate_code(problem) 
    for problem in problems
])

# Create examples
examples = []
for i, (problem, generated_code) in enumerate(zip(problems, generated_codes)):
    print(f"   Problem {i+1}/5: {problem['task_id']}")
    
    example = Example(
        task_id=problem["task_id"],
        prompt=problem["prompt"],
        canonical_solution=problem["canonical_solution"],
        test=problem["test"],
        entry_point=problem["entry_point"],
        generated_code=generated_code
    )
    examples.append(example)


🤖 Generating code...
   Problem 1/5: HumanEval/0
   Problem 2/5: HumanEval/1
   Problem 3/5: HumanEval/2
   Problem 4/5: HumanEval/3
   Problem 5/5: HumanEval/4
   Problem 6/5: HumanEval/5
   Problem 7/5: HumanEval/6
   Problem 8/5: HumanEval/7
   Problem 9/5: HumanEval/8
   Problem 10/5: HumanEval/9
   Problem 11/5: HumanEval/10
   Problem 12/5: HumanEval/11
   Problem 13/5: HumanEval/12
   Problem 14/5: HumanEval/13
   Problem 15/5: HumanEval/14
   Problem 16/5: HumanEval/15
   Problem 17/5: HumanEval/16
   Problem 18/5: HumanEval/17
   Problem 19/5: HumanEval/18
   Problem 20/5: HumanEval/19
   Problem 21/5: HumanEval/20
   Problem 22/5: HumanEval/21
   Problem 23/5: HumanEval/22
   Problem 24/5: HumanEval/23
   Problem 25/5: HumanEval/24
   Problem 26/5: HumanEval/25
   Problem 27/5: HumanEval/26
   Problem 28/5: HumanEval/27
   Problem 29/5: HumanEval/28
   Problem 30/5: HumanEval/29
   Problem 31/5: HumanEval/30
   Problem 32/5: HumanEval/31
   Problem 33/5: HumanEval/32
   Prob

We use `Dataset.create()` to create a new dataset and upload it to the judgment platform.

In [7]:
dataset = Dataset.create(
    name="humaneval-dataset", 
    project_name="humaneval-project", 
    examples=examples,
    overwrite=True
)

2025-09-18 20:20:15 - judgeval - INFO - Successfully created dataset humaneval-dataset!


We use `Dataset.get()` to retrieve an existing dataset from the judgment platform.

In [8]:
dataset = Dataset.get(
    name="humaneval-dataset",
    project_name="humaneval-project"
)

2025-09-18 20:20:16 - judgeval - INFO - Successfully retrieved dataset humaneval-dataset!


## Running Evaluation

We then use our custom scorer and the judgment client to run evaluations asynchronously on our servers and display the results on the judgment platform for analysis.

In [9]:
print("\n⚡ Running evaluation...")
judgment.run_evaluation(
    examples=dataset.examples,
    scorers=[HumanEvalCodeExecutionScorer()],
    project_name="humaneval-project"
)


⚡ Running evaluation...


Evaluating 164 example(s) in parallel: |██████████|100% (164/164) [Time Taken: 00:10, 15.24Example/s]


🔍 You can view your evaluation results here: ]8;id=150514;https://app.judgmentlabs.ai/org/12b03546-81a4-4f6c-b3cf-4670c9b8548e/project/cc6a4ad8-b66d-4980-bd7c-fdd056c88c71/tests/5441c993-28e2-498c-b27a-0f5fa59009cd\View Results]8;;\

[ScoringResult(success=True, scorers_data=[ScorerData(id=None, name='HumanEval Code Execution Scorer', threshold=0.5, success=True, score=1.0, reason='All test cases passed', strict_mode=False, evaluation_model=None, error=None, additional_metadata=None)], name=None, data_object=Example(example_id='085d05f5-f7d8-4b3e-a6f2-b0e5d64c9d4c', created_at='2025-09-18T20:20:13.21502+00:00', name=None, dataset_id='68719bf9-168f-4209-b255-ccf4dbe24a49', test="\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\